In [45]:
import typing

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions
import sentence_transformers
import util
import numpy as np

In [3]:
Model = sentence_transformers.SentenceTransformer("E:\models\BAAI_bge-base-zh-v1.5").cuda().half()

In [84]:
encode_ef = Model.encode
chroma_client = chromadb.PersistentClient()

In [55]:

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        return encode_ef(input).tolist()

In [93]:
embeddings = encode_ef([
    "你好啊，世界！",
    "世界，你好啊！",
    "水调歌头·明月几时有",
    "苏轼的词《水调歌头》全文",
    """
    丙辰中秋，欢饮达旦，大醉，作此篇，兼怀子由。 \n\n明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间？\n转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。\n\n

    """,
    "但愿人长久，千里共婵娟"
                         
])

In [99]:
collection = chroma_client.get_or_create_collection(name="test2",
                                                    embedding_function=MyEmbeddingFunction(),
                                                    metadata={"hnsw:space": "ip"})

In [96]:
import util
util.dot_product(embeddings[5],embeddings[1])

0.4219

In [100]:
collection.add(
    ids=["1","2","3","4","5","6"],
    embeddings=embeddings.tolist(),
    metadatas=[{"key":"你好啊，世界！"},
               {"key":"世界，你好啊！"},
               {"key":"水调歌头·明月几时有"},
               {"key":"苏轼《水调歌头》全文"},
               {"key":"""
    丙辰中秋，欢饮达旦，大醉，作此篇，兼怀子由。 \n\n明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间？\n转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。\n\n

    """,},
               {"key":"但愿人长久，千里共婵娟"}],
)

Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6


In [101]:
results = collection.query(query_texts="但愿人长久，千里共婵娟",n_results=4)
results

{'ids': [['6', '5', '2', '4']],
 'distances': [[-2.3865336444828245e-06,
   0.36458637230225577,
   0.5779726172809205,
   0.5792259860263584]],
 'metadatas': [[{'key': '但愿人长久，千里共婵娟'},
   {'key': '\n    丙辰中秋，欢饮达旦，大醉，作此篇，兼怀子由。 \n\n明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间？\n转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。\n\n\n\n    '},
   {'key': '世界，你好啊！'},
   {'key': '苏轼《水调歌头》全文'}]],
 'embeddings': None,
 'documents': [[None, None, None, None]],
 'uris': None,
 'data': None}